# Л2: Методы обработки изображений

**CV (Computer Vision)**, или *машинное зрение*, — это область искусственного интеллекта, которая занимается разработкой алгоритмов и методов, позволяющих компьютерам "видеть" и интерпретировать изображения и видео. Основная задача компьютерного зрения — автоматическое извлечение полезной информации из визуальных данных, например, распознавание объектов, определение их местоположения или анализ сцен. Это может включать такие приложения, как автоматизация вождения, распознавание лиц и обработка медицинских изображений.

**OpenCV (Open Source Computer Vision Library)** — библиотека алгоритмов компьютерного зрения, обработки изображений и численных алгоритмов общего назначения с открытым кодом

## ☝️ПОРЯДОК ВЫПОЛНЕНИЯ

ЛР выполняется последователньно. По мере продвижения по работе вы будете получать задания для закрепления прочитанной теории

✏️ **ЗАДАНИЕ 1.**
✏️ **ЗАДАНИЕ 2.** и т.д.

Для каждого задания создается отдельный скрипт Python. Всего заданий будет 4. В качестве отчета необходимо предоставить файлы кода для каждого задания. Отчет закреплять на moodle.

## 1/ Подготовка среды выполнения
На данной этапе вам наобходимо подготовить виртуальное окружение и установить все необходимые библиотеки.

1. Создать и активировать (или только активировать, если ранне создавали) виртуальной окружение `python`.

Создайте на диске свою раюочую папку. Откройте терминал и перейдите в свою рабочую директорию
```
cd path/to/your/workspace
```
Далее создаем виртуальное окружение с помощью `python-venv`
```
python -m venv env
```
*активируем виртуальное окружение*
для CMD:
```
env\Scripts\activate
```
для PowerShell
```
env\Scripts\Activate.ps1
```
для bash
```
source env/bin/activate
```
**Примечание.** `env` - это название вашего виртуального окружения, назвать его можете как угодно.

После этого можем выбрать наш локальный интерпрететор pyhton, нажав на кнопку выше "Select kernel".

1. Устанавливаем все необходимые библиотеки

**Примечание.** Библиотеки установятся внутрь вашего виртуального окружения.

Нам понадобятся opencv-python.
```
pip install opencv-python
```

## 2/ Получение видеопотока

Вы можете использовать три варианта получения видеопотока: **из локальной сети на занятии в аудитории, со встроенной или подключенной камере к вашему ноутбуку или из видеофайла.**

### 1. Видеокамера (встроенная в ноутбук или внешняя)
Открытие видеопотока с камеры:
```python
# Захватываем видеопоток с вебкамеры
cap = cv2.VideoCapture(0) # или /dev/video0 на Linux
...
ret, frame = cap.read() # чтение кадра из видеопотока
```

### 2. Видеофайл на диске
Открытие видеопотока из файла:
```python
# Захватываем видеопоток из файла
cap = cv2.VideoCapture('path/to/video.mp4')
...
ret, frame = cap.read() # чтение кадра из видеопотока
```

### 3. Видеопоток по локальной сети (в условиях лаборатории)
Для получения видеопотока по сети необходимо создать UDP сокет и подключится к multicast группе. В главном цикле читается байтовый массив по сокету, который преобразуется в jpeg-кадр. Ниже представлен базоый скрипт клиента получения видеопотока:

```python
import socket
import cv2
import numpy as np

# Настройки multicast
MCAST_GRP = '239.1.1.1'
MCAST_PORT = 5007

############### ВМЕСТО cap = cv2.VideoCapture(0) ####################
#####################################################################
# Создаем UDP сокет
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Присоединяемся к multicast группе
sock.bind(('', MCAST_PORT))
mreq = socket.inet_aton(MCAST_GRP) + socket.inet_aton('0.0.0.0')
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)
#####################################################################

# Главный цикл
while True:
############### ВМЕСТО ret, frame = cap.read() ######################
#####################################################################
    # Получаем данные
    data, addr = sock.recvfrom(65536)  # Макс размер UDP пакета

    # Декодируем JPEG в изображение
    nparr = np.frombuffer(data, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#####################################################################

    if frame is not None:s
        cv2.imshow('Multicast Stream', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
sock.close()
```

## 3/ Методы обработки изображений
### 3.1/ Преобразование между цветовыми пространствами

![color_models](../images/color_models.png)

Преобразование между цветовыми пространствами в компьютерном зрении - это изменение представления цветов пикселей из одной системы в другую. Существуют множество различных цветовых пространст. Ниже приведены некоторые из наиболее распространенных.

**RGB (Red, Green, Blue)** – одно из самых широко используемых цветовых пространств в компьютерной графике и фотографии. Каждый цвет представлен комбинацией трех основных цветов: красного, зеленого и синего. Максимальные значения каждого цвета обычно равны 255.

Прямой аналогией получения различных оттенков в этом цветовом пространстве является, например, смешивания масляных красок.

RGB используется в основном для отображения цветов на электронных устройствах, таких как мониторы компьютеров, телевизоры и смартфоны.

---

**CMYK (Cyan, Magenta, Yellow, Key/Black)** используется в печати и изображениях. Он основан на комбинации четырех основных цветов: голубого (cyan), пурпурного (magenta), желтого (yellow) и черного (key/black). Это цветовое пространство позволяет точнее представлять цвета, которые можно воспроизвести на печатной машине. Однако, CMYK не может отобразить весь спектр цветов, доступный в RGB. Перевод из RGB в CMYK часто приводит к потере точности цветопередачи.

CMYK преимущественно используется в полиграфии для печати книг, журналов, рекламной продукции, упаковки и широкоформатных материалов, обеспечивая точное воспроизведение цветов на печатном оборудовании.

---

**HSV (Hue, Saturation, Value)** представляет цвета с помощью трех компонентов: оттенок (hue), насыщенность (saturation) и значение (value). Он предоставляет удобный способ управления цветом, особенно для графического дизайна.

---

**LAB (CIELAB)** – это цветовая модель, которая разработана для описания всего видимого спектра цветов. Она была разработана Международной комиссией по освещению (CIE) в 1976 году.

Цветовое пространство LAB разделяет цвет на светлоту и две цветовые оси: (зелёный – красный) и (синий – жёлтый). Это пространство более пригодно для восприятия человеческим глазом и широко используется в приложениях, связанных с визуальным восприятием.


LAB основано на принципах цветового восприятия человеческого глаза, что делает его более естественным для восприятия человеком. Он учитывает особенности восприятия яркости, контраста и насыщенности, что делает его более соответствующим тому, как люди видят и воспринимают цвета.

Цветовое пространство LAB используется в профессиональной обработке изображений, цветокоррекции и профилировании, где требуется высокая точность цветопередачи и независимость от устройства вывода. Его применяют в фотопечати, программном обеспечении для управления цветом и в системах контроля качества цветовоспроизведения.

---

Цветовое пространство **BGR (Blue, Green, Red)** используется в компьютерной графике и обработке изображений. Оно представляет собой способ организации цветовой информации, где каждый пиксель изображения определяется тремя компонентами: синим (Blue), зеленым (Green) и красным (Red).

BGR преимущественно используется в компьютерной графике и обработке изображений, особенно в программировании и разработке графических приложений. Это цветовое пространство часто встречается в работе с изображениями в формате BGR, который является одним из стандартных форматов хранения изображений в компьютерных системах. Он также широко применяется в библиотеках и инструментах для обработки изображений, таких как **OpenCV** в языке программирования **Python**.

Технический аспект заключается в том, что многие камеры и библиотеки обработки изображений в настоящее время продолжают использовать **BGR**, чтобы сохранять совместимость со старым программным обеспечением и форматами данных. В **OpenCV** BGR также используется по умолчанию при чтении изображений с помощью функций библиотеки.

☝️Важно помнить о порядке цветов (**BGR**), когда вы работаете с изображениями в **OpenCV**, чтобы избежать недопонимания при обработке и отображении цветовых данных.